In [1]:
import pandas as pd
import numpy as np

# Setup

Vamos a usar el dataset de Titanic que podemos descargar desde la página de [Kaggle](https://www.kaggle.com/c/titanic/data).

Las variables son: 
* survival: si sobrevivió o no (boolean)
* pclass: clase
* sex: género
* Age: edad
* sibsp: cantidad de hermanos o pareja
* parch: cantidad de padres o hijos
* ticket: numero de ticket
* fare: tarifa
* cabin: numero de cabina
* embarked: puerto en el cual se embarcó

Este es un dataset comunente usado para modelos de machine learning que buscan predecir si el pasajero sobrevivió o no el accidente. Sin embargo, antes de comenzar, es necesario hacer una limpieza de los datos para eliminar registros que no serán de utilidad.

In [2]:
titanic = pd.read_csv('../DataSets/titanic.csv')
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# Vamos a conservar un subset de variables para el proposito del ejercicio de duplicados
df = titanic.loc[:, ['Survived', 'Pclass', 'Sex', 'Cabin', 'Embarked']]

In [4]:
df.describe()
#df.shape

,Survived,Pclass
count,891.000000,891.000000
mean,0.383838,2.308642
std,0.486592,0.836071
min,0.000000,1.000000
25%,0.000000,2.000000
50%,0.000000,3.000000
75%,1.000000,3.000000
max,1.000000,3.000000


In [5]:
df.groupby('Survived').agg({'Survived':'count'})

,Survived
Survived,
0,549
1,342


# 1. Duplicados

## 1.1 Buscamos las filas duplicadas

Primero vamos a identificar los registros duplicados.

In [6]:
# veamos la variable Cabin
df["Cabin"].duplicated()

0      False
1      False
2       True
3      False
4       True
       ...  
886     True
887    False
888     True
889    False
890     True
Name: Cabin, Length: 891, dtype: bool

El resultado son un conjunto de valores True y False. Aquellos que tienen True, son casos duplicados (ya aparecieron antes). 

Veamos si quisieramos ver casos duplicados al nivel de todas las variables.

In [7]:
df.duplicated()

0      False
1      False
2      False
3      False
4       True
       ...  
886     True
887    False
888     True
889    False
890     True
Length: 891, dtype: bool

También podriamos querer ver los registros duplicados solo para un grupo de variables.

In [8]:
df.duplicated(subset=['Survived', 'Pclass', 'Sex'])

0      False
1      False
2      False
3       True
4       True
       ...  
886     True
887     True
888     True
889     True
890     True
Length: 891, dtype: bool

## 1.2 Contabilizar los duplicados

Si bien con los ejemplos anteriores pudimos identificar los duplicados, no nos sirve de mucho para dimensionar el problema. Veamos como podemos hacerlo.

In [9]:
# conteo de duplicados en una columna
df.Cabin.dropna().duplicated().sum() # ignoro los faltantes

57

In [10]:
# conteo de duplicados en el dataset 
df.duplicated().sum()

688

In [11]:
# conteo de duplicados para un grupo de variables
df.duplicated(subset=['Survived', 'Pclass', 'Sex']).sum()

879

No deberia soprendernos que sean tantos ya que para esa combinación de variables existen solamente 12 combinaciones únicas.

Veamos ahora como contar los casos no duplicados.

## 1.3 Eliminamos duplicados

In [12]:
# extraemos las filas duplicadas
df.loc[df.duplicated(), :]

,Survived,Pclass,Sex,Cabin,Embarked
4,0,3,male,NaN,S
7,0,3,male,NaN,S
8,1,3,female,NaN,S
12,0,3,male,NaN,S
13,0,3,male,NaN,S
...,...,...,...,...,...
884,0,3,male,NaN,S
885,0,3,female,NaN,Q
886,0,2,male,NaN,S
888,0,3,female,NaN,S


In [13]:
# extraemos las filas no duplicadas
df.loc[(~df.duplicated()), :]

,Survived,Pclass,Sex,Cabin,Embarked
0,0,3,male,NaN,S
1,1,1,female,C85,C
2,1,3,female,NaN,S
3,1,1,female,C123,S
5,0,3,male,NaN,Q
...,...,...,...,...,...
871,1,1,female,D35,S
872,0,1,male,B51 B53 B55,S
879,1,1,female,C50,C
887,1,1,female,B42,S


## 1.4 Seleccion de variables a conservar

Elegimos si queremos, dentro de los duplicados, conservar el primer registro que aparece o el ultimo.

In [14]:
df.loc[df.duplicated(keep='first'), :]

,Survived,Pclass,Sex,Cabin,Embarked
4,0,3,male,NaN,S
7,0,3,male,NaN,S
8,1,3,female,NaN,S
12,0,3,male,NaN,S
13,0,3,male,NaN,S
...,...,...,...,...,...
884,0,3,male,NaN,S
885,0,3,female,NaN,Q
886,0,2,male,NaN,S
888,0,3,female,NaN,S


In [15]:
df.loc[df.duplicated(keep='last'), :]

,Survived,Pclass,Sex,Cabin,Embarked
0,0,3,male,NaN,S
2,1,3,female,NaN,S
4,0,3,male,NaN,S
5,0,3,male,NaN,Q
7,0,3,male,NaN,S
...,...,...,...,...,...
877,0,3,male,NaN,S
878,0,3,male,NaN,S
881,0,3,male,NaN,S
882,0,3,female,NaN,S


In [16]:
# Existe una tercera opcion que nos permite ver todos los duplicados
df.loc[df.duplicated(keep=False), :]

,Survived,Pclass,Sex,Cabin,Embarked
0,0,3,male,NaN,S
2,1,3,female,NaN,S
4,0,3,male,NaN,S
5,0,3,male,NaN,Q
7,0,3,male,NaN,S
...,...,...,...,...,...
884,0,3,male,NaN,S
885,0,3,female,NaN,Q
886,0,2,male,NaN,S
888,0,3,female,NaN,S


## 1.5 Eliminar duplicados

Veamos la opción de eliminar duplicados.

In [17]:
df.drop_duplicates()

,Survived,Pclass,Sex,Cabin,Embarked
0,0,3,male,NaN,S
1,1,1,female,C85,C
2,1,3,female,NaN,S
3,1,1,female,C123,S
5,0,3,male,NaN,Q
...,...,...,...,...,...
871,1,1,female,D35,S
872,0,1,male,B51 B53 B55,S
879,1,1,female,C50,C
887,1,1,female,B42,S


In [18]:
# conservamos el ultimo caso duplicado 
df.drop_duplicates(keep='last')

,Survived,Pclass,Sex,Cabin,Embarked
1,1,1,female,C85,C
3,1,1,female,C123,S
6,0,1,male,E46,S
11,1,1,female,C103,S
21,1,2,male,D56,S
...,...,...,...,...,...
886,0,2,male,NaN,S
887,1,1,female,B42,S
888,0,3,female,NaN,S
889,1,1,male,C148,C


In [19]:
# conservamos el primer caso duplicado
df.drop_duplicates(keep='first')

,Survived,Pclass,Sex,Cabin,Embarked
0,0,3,male,NaN,S
1,1,1,female,C85,C
2,1,3,female,NaN,S
3,1,1,female,C123,S
5,0,3,male,NaN,Q
...,...,...,...,...,...
871,1,1,female,D35,S
872,0,1,male,B51 B53 B55,S
879,1,1,female,C50,C
887,1,1,female,B42,S


In [20]:
# si queremos eliminar los duplicados modificando directamente el dataframe usamos el argumento inplace
# df.drop_duplicates(inplace=True)

También tenemos la opción de eliminar registros si se repiten valores en una unica variable.

In [21]:
df.drop_duplicates(subset=['Survived', 'Pclass', 'Sex'])

,Survived,Pclass,Sex,Cabin,Embarked
0,0,3,male,NaN,S
1,1,1,female,C85,C
2,1,3,female,NaN,S
6,0,1,male,E46,S
9,1,2,female,NaN,C
14,0,3,female,NaN,S
17,1,2,male,NaN,S
20,0,2,male,NaN,S
23,1,1,male,A6,S
36,1,3,male,NaN,C


# 2. Faltantes

## 2.1 Identificar faltantes

Lo primero que tenemos que hacer es identificar los faltantes de nuestro dataset. Para esta sección vamos a usar todo el dataset

In [22]:
df = titanic

In [23]:
# para una sola variable
df['Age'].isnull()

0      False
1      False
2      False
3      False
4      False
       ...  
886    False
887    False
888     True
889    False
890    False
Name: Age, Length: 891, dtype: bool

In [24]:
# para todo el dataset 
df.isnull()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,False,False,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,True,False
3,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
886,False,False,False,False,False,False,False,False,False,False,True,False
887,False,False,False,False,False,False,False,False,False,False,False,False
888,False,False,False,False,False,True,False,False,False,False,True,False
889,False,False,False,False,False,False,False,False,False,False,False,False


## 2.2 Contabilizar faltantes
Nuevamente, nos va a interesar dimensionar el problema de los faltantes en nuestro dataset.

In [25]:
df['Age'].isnull().sum()

177

In [26]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [27]:
df.isnull().mean() * 100

PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            19.865320
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
Cabin          77.104377
Embarked        0.224467
dtype: float64

In [28]:
df.isnull().sum() / df.shape[0]

PassengerId    0.000000
Survived       0.000000
Pclass         0.000000
Name           0.000000
Sex            0.000000
Age            0.198653
SibSp          0.000000
Parch          0.000000
Ticket         0.000000
Fare           0.000000
Cabin          0.771044
Embarked       0.002245
dtype: float64

## 2.3 Eliminar faltantes
Podemos eliminar aquellas filas que contienen datos faltantes.

In [29]:
df.dropna()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S
...,...,...,...,...,...,...,...,...,...,...,...,...
871,872,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.0,1,1,11751,52.5542,D35,S
872,873,0,1,"Carlsson, Mr. Frans Olof",male,33.0,0,0,695,5.0000,B51 B53 B55,S
879,880,1,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,0,1,11767,83.1583,C50,C
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S


Sin embargo esto puede traernos algunos problemas si elimnamos  muchos registros. También tenemos la opción de eliminar filas con faltantes en alguna variable.

In [30]:
df.loc[df['Age'].notna(), :]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
885,886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.1250,NaN,Q
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


Imaginemos ahora si queremos conservar solo los que tienen faltantes en una fila

In [31]:
df[df['Age'].isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,S
19,20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,NaN,C
26,27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,NaN,C
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,NaN,Q
...,...,...,...,...,...,...,...,...,...,...,...,...
859,860,0,3,"Razi, Mr. Raihed",male,NaN,0,0,2629,7.2292,NaN,C
863,864,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,NaN,8,2,CA. 2343,69.5500,NaN,S
868,869,0,3,"van Melkebeke, Mr. Philemon",male,NaN,0,0,345777,9.5000,NaN,S
878,879,0,3,"Laleff, Mr. Kristo",male,NaN,0,0,349217,7.8958,NaN,S


## 2.4 Imputación de faltantes


### 2.4.1 Valor homogéneo

Podemos simplemente imputar un unico valor para los faltantes. Antes de hacerlo, debemos analizar si tiene sentido y coherencia con la información de la variable. La función recomendada es **fillna()**.

In [32]:
df_sna = df.copy()

# podemos hacerlo usando el comando replace
df_sna['Age'] = df_sna['Age'].replace(np.nan, 0)
df_sna['Cabin'] = df_sna['Cabin'].replace(np.nan, 'Sin Dato')

df_sna.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       2
dtype: int64

In [33]:
df_sna = df.copy()

# o el comando fillna
df_sna['Age'] = df_sna['Age'].fillna(0)
df_sna['Cabin'] = df_sna['Cabin'].fillna('Sin Dato')

df_sna.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       2
dtype: int64

### 2.4.2 Valor medio
También podemos imputar con algun valor que se desprenda de los datos que tenemos de la variable. Por ejemplo, para imputar la edad podemos considerar el promedio de las edades e imputar ese valor.

In [34]:
df['Age'].mean()

29.69911764705882

In [35]:
df['Age_2'] = df['Age'].fillna(df['Age'].mean())
# df['Age'].fillna(df['Age'].mean(), inplace = True)

In [36]:
df['Age_2'].isnull().sum()

0

También podríamos imputar teniendo en cuenta algun tipo de desagregación del valor medio, como por ejemplo la clase del pasajero.

In [37]:
df.groupby('Pclass').agg({'Age': 'mean'})

,Age
Pclass,
1,38.233441
2,29.877630
3,25.140620


In [38]:
df.groupby('Pclass')['Age'].transform('mean')

0      25.140620
1      38.233441
2      25.140620
3      38.233441
4      25.140620
         ...    
886    29.877630
887    38.233441
888    25.140620
889    38.233441
890    25.140620
Name: Age, Length: 891, dtype: float64

In [39]:
df['Age_3'] = df['Age'].fillna(
    df.groupby('Pclass')['Age'].transform('mean')
    )

Comparemos los inputs.

In [40]:
df.loc[~df['Age'].notna(), ['Pclass','Age', 'Age_2', 'Age_3']]

,Pclass,Age,Age_2,Age_3
5,3,NaN,29.699118,25.14062
17,2,NaN,29.699118,29.87763
19,3,NaN,29.699118,25.14062
26,3,NaN,29.699118,25.14062
28,3,NaN,29.699118,25.14062
...,...,...,...,...
859,3,NaN,29.699118,25.14062
863,3,NaN,29.699118,25.14062
868,3,NaN,29.699118,25.14062
878,3,NaN,29.699118,25.14062


### 2.4.3 Regresión
Una última opción es imputar los valores usando algún mecanismo más sofisticado, como por ejemplo una regresión que en base a los datos disponibles del pasajer prediga el valor de la edad.

Este la implementación de este método excede los alcances del texto, pero es importante mencionarlo para tenerlo presente.

# 3. Sanity Check

Imaginemos que queremos hacer un análisis sobre las tarifas pagas por los pasajeros.

En primer lugar nos interesaría que esta variable este completa y que no tenga valores en 0.

In [41]:
# Sabemos que no tiene faltantes de los analisis previos, por eso eliminamos aquellos casos que valen 0
df_p = df[df.Fare != 0]
len(df_p)

876

Analicemos los precios pagos en cada clase.

In [42]:
df_p.groupby('Pclass').agg({'Fare': ['mean', 'min', 'max']})

Fare                   
             mean      min       max
Pclass                              
1       86.148874   5.0000  512.3292
2       21.358661  10.5000   73.5000
3       13.787875   4.0125   69.5500

Vemos que existe varias inconsistencias en tanto que es posible que un pasajero de primera clase haya pagado menos que uno de tercera y, por el contrario, que uno de tercera haya pagado más que uno de primera.

Si quisieramos usar la variable de la clase del pasajero para intentar predecir cuánto pagó por su pasaje, tendríamos un problema. 

Si se tuviera disponible un rango de precios de los pasajes, se podría corregir la variable de la clase del pasajero.